In [ ]:
import pandas as pd
from datetime import datetime, date
import numpy as np
import os

In [190]:
# Limito la lectura del Padron utilizando solo las columnas necesarias
cols = ['N°', 'NOMBRE', 'Fecha apertura', 'ORGANIZACIÓN ', 'DIRECTOR EXPLOTACIÓN', 'DIRECTOR OPERACIONAL', 'DIRECTOR / GERENTE REGIONAL', 'DIRECTOR/ GERENTE TIENDA', 'M² SALÓN', 'M² PGC', 'M² PFT', 'M² BAZAR', 'M² Electro', 'M² Textil', 'M² Pls', 'M² GALERIAS', 'DOMICILIO', 'PROVINCIA', 'CAJAS', 'M² Parcking', 'FIN DE CIERRE', 'ENE.2', 'FEB.2', 'MAR.2', 'ABR.2', 'MAY.2', 'JUN.2', 'JUL.2', 'AGO.2', 'SEP.2', 'OCT.2', 'NOV.2', 'DIC.2']

# Leo el archivo (Conteplar que el archivo debe estar dentro de una carpeta llamada data)
padron = pd.read_excel('data/padron.xlsx', header=17, usecols=cols)

# Normalizo los nombres de las columnas
padron.columns = padron.columns.str.lower().str.replace('/', '').str.replace('.2', '').str.strip().str.replace('  ', ' ').str.replace(' ', '_').str.replace('m²', 'mtr')
padron = padron.rename(columns={'n°':'numero_operacional'})

# Elimino las filas que tengan na en su numero, nombre, organizacion
padron.dropna(subset=['numero_operacional', 'nombre', 'organización'], how='any')

# Obtengo unicamente los valores que tengan Numero Operacional y no tengan nulos
padron = padron[(padron['numero_operacional'] != '-') & (padron['numero_operacional'].notna())]

# Convierto la Columna a INT
padron['numero_operacional'] = pd.to_numeric(padron['numero_operacional'], errors='coerce').astype('Int64')

# Me quedo con aquellas lineas que tengan numero Operacional
padron = padron[padron['numero_operacional'].notna()]

# Replazo valores nullos por dummies
padron['fecha_apertura'] = padron['fecha_apertura'].fillna(date(2020, 1, 1))

# Realizo una seleccion de columnas
columnas_categoricas = ['director_explotación', 'director_operacional', 'director_gerente_regional', 'director_gerente_tienda', 'domicilio', 'provincia', 'ene', 'feb', 'mar', 'abr', 'may', 'jun', 'jul', 'ago', 'sep', 'oct', 'nov', 'dic']
date_columns = ['fecha_apertura', 'fin_de_cierre']
num_columns = ['mtr_salón', 'mtr_pgc', 'mtr_pft', 'mtr_bazar', 'mtr_electro', 'mtr_textil', 'mtr_pls', 'mtr_galerias', 'cajas', 'mtr_parcking']
string_columns = ['nombre', 'organización', 'director_explotación', 'director_operacional', 'director_gerente_regional', 'director_gerente_tienda', 'domicilio', 'provincia', 'ene', 'feb', 'mar', 'abr', 'may', 'jun', 'jul', 'ago', 'sep', 'oct', 'nov', 'dic']

# Normalizo algunos valores de las columnas Datetime
padron['fin_de_cierre'] = padron['fin_de_cierre'].replace('-', '')
padron['fin_de_cierre'] = padron['fin_de_cierre'].fillna('')

# Itero sobre las columnas seleccionadas anteriormente para completar, limpiar y normalizar valores. Esto evita errores cuando se convierten a su tipo de datos indicado
for col in columnas_categoricas:
    padron[col] = padron[col].fillna('S/D')

for col in num_columns:
    padron[col] = padron[col].fillna(0)
    padron[col] = padron[col].replace('sd', 0).replace('SD', 0).replace('-', 0)
    padron[col] = padron[col].astype(int)

for col in string_columns:
    padron[col] = padron[col].str.lower().astype(str)

for col in date_columns:
    padron[col] = pd.to_datetime(padron[col])

# Genero una nueva columna condicional que identifica cuales son las tiendas cerradas en base a su fecha de cierre, cuales estan activas en base a a la falta de una fecha de cierre, y aquellas que estan por abrir, comparando su fecha de apertura con la fecha en la cual corre el script
padron['estado_tienda'] = np.where(padron['fecha_apertura'].dt.date > date(datetime.today().year, datetime.today().month, datetime.today().day), 'proxima apertura', np.where(padron['fin_de_cierre'].notna(), 'cerrada', 'activa'))

# Finalmente renombro columnas para que no lleven tilde
padron = padron.rename(columns={
    'organización':'organizacion',
    'director_explotación':'director_explotacion',
    'mtr_salón':'mtr_salon'
})

C:\Users\Juan Mera\AppData\Local\Temp\ipykernel_5044\2011166845.py:42: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  padron[col] = padron[col].replace('sd', 0).replace('SD', 0).replace('-', 0)
C:\Users\Juan Mera\AppData\Local\Temp\ipykernel_5044\2011166845.py:42: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  padron[col] = padron[col].replace('sd', 0).replace('SD', 0).replace('-', 0)
C:\Users\Juan Mera\AppData\Local\Temp\ipykernel_5044\2011166845.py:42: FutureWarning: Downcasting behavior in `replace` is deprecated and will be remo